# P02: Introdução ao Spark com Python

## RDD: `map`, `filter` e `collect`

## Como obter o `SparkContext`

O `SparkContext` Carregado automaticamente quando o notebook é iniciado pelo PySpark

In [ ]:
# Somente necessário quando o notebook não é carregado pelo PySpark
#import pyspark
#sc = pyspark.SparkContext(appName="P2")

# Quando o notebook é carregado pelo PySpark, a variável sc é disponibilizada automaticamente
sc

## Obtendo o conjunto de dados de análise reduzido

Usaremos um conjunto reduzido de dados (10%) da Copa KDD de 1999, que contém quase meio milhão de registros

O arquivo é fornecido como um *Gzip*

In [ ]:
import urllib.request as request
f = request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

Atenção! Lembre-se de colocar o arquivo baixado no `HDFS` (caso ainda não tenha feito). Além disso, inicie o `HDFS` e o `Yarn`.

```bash
start-dfs.sh
start-yarn.sh
hdfs dfs -put kddcup.data_10_percent.gz /usr/hduser
```

Podemos usar esse arquivo para criar nosso RDD.

In [ ]:
nome_arquivo = "./kddcup.data_10_percent.gz"
rdd = sc.textFile(nome_arquivo)

## A transformação `filter`

Essa transformação pode ser aplicada aos RDDs para manter apenas os elementos que satisfazem uma determinada condição. Mais especificamente, uma função é avaliada em cada elemento no RDD original. O RDD resultante conterá apenas os elementos que fazem a função retornar `True`.

Por exemplo, imagine que queremos contar quantas interações `normais` que temos em nosso conjunto de dados. Podemos filtrar nosso RDD da seguinte maneira.

In [ ]:
normal_rdd = rdd.filter(lambda x: 'normal.' in x)

Agora podemos contar quantos elementos temos no novo RDD.

In [ ]:
from time import time
t0 = time()
contagem_normal = normal_rdd.count()
tt = time() - t0
print("Existem {} interações 'normais'".format(contagem_normal))
print("Contagem concluída em {} segundos".format(round(tt,3)))

Lembre-se que temos um total de 494.021 em nosso conjunto de dados. Podemos ver que 97.278 contém a palavra `normal`.

Observe que medimos o tempo decorrido para contar os elementos no RDD. Fizemos isso porque queríamos apontar que computações reais (distribuídas) no Spark acontecem quando executamos *ações* e não *transformações*. 

## A transformação `map`

Usando a transformação `map` no Spark, podemos aplicar uma função a todos os elementos do nosso RDD.

Neste caso, queremos ler nosso arquivo de dados como um arquivo formatado em CSV. Podemos fazer isso aplicando uma função lambda a cada elemento no RDD da seguinte maneira.

In [ ]:
from pprint import pprint
csv_dados = rdd.map(lambda x: x.split(","))
t0 = time()
elementos = csv_dados.take(5)
tt = time() - t0
print("Ação completada em {} segundos".format(round(tt,3)))
pprint(elementos[0])

E se pegarmos muitos elementos em vez de apenas os primeiros?

In [ ]:
t0 = time()
elementos = csv_dados.take(100000)
tt = time() - t0
print("Ação completada em {} segundos".format(round(tt,3)))

Demora mais tempo. Isso porque a função `map` é aplicada agora de maneira distribuída a muitos elementos no RDD.

### Usando o `map` e funções predefinidas

Claro que podemos usar funções pré-definidas com o `map`. Imagine que queremos ter cada elemento no RDD como um par de valores-chave em que a chave é a tag (por exemplo, *normal*) e o valor é toda a lista de elementos que representa a linha no arquivo CSV. Nós poderíamos proceder da seguinte forma.

In [ ]:
def em_chave_valor(linha):
    todos_itens = linha.split(",")
    chave = todos_itens[41]
    return (chave, todos_itens)

chave_valor_csv = rdd.map(em_chave_valor)
elementos = chave_valor_csv.take(5)
pprint(elementos[0])

## A ação `collect`

A ação `collect` irá colocar todos os elementos do RDD na memória. Por esse motivo, ele deve ser usado com cuidado, especialmente quando se trabalha com grandes RDDs.

In [ ]:
t0 = time()
todos_elementos = rdd.collect()
tt = time() - t0
print("Dados coletados em {} segundos".format(round(tt,3)))

Note que esse processamento demorou mais que as outras ações. Cada nó de processamento do Spark que possui uma parte do RDD precisa de gerenciamento na recuperação das informações e então 'juntar' `reduce` tudo novamente.

Como último exemplo que combina todos os anteriores, queremos coletar todas as interações `normal` como pares de valor-chave.

In [ ]:
# Obtém os dados de um arquivo
nome_arquivo = "./kddcup.data_10_percent.gz"
meu_rdd = sc.textFile(nome_arquivo)

# transforma em chave valor
chave_valor_rdd = meu_rdd.map(em_chave_valor)

# filtra por interações normais
normais_rdd = chave_valor_rdd.filter(lambda x: x[0] == "normal.")

# coleta tudo e calcula o tempo
t0 = time()
normais = normais_rdd.collect()
tt = time() - t0
contagem_normais = len(normais)

print("Dados coletados em {} segundos".format(round(tt,3)))
print("Existem {} interações 'normal'".format(contagem_normais))

Esta contagem corresponde à contagem anterior para interações `normal`. O novo procedimento é mais demorado. Isso ocorre porque recuperamos todos os dados com `collect` e usamos o `len` do Python na lista resultante. Antes estávamos contando apenas o número total de elementos no RDD usando `count`.